# ***Espero não ter abusado do uso dos comentários.***
#### **Decidi usar para melhor explicar ao professor, na hora de entender minha lógica usada.**

# ***Necessidades do projeto***

In [1]:
#Necessidades.
# No terminal, use: pip install requests
# No terminal, use: pip install pandas
# No terminal, use: pip install plyer



import requests
import pandas as pd
from plyer import notification
from datetime import datetime

# ***Criando as funções***

In [11]:
# Notificação
def alerta(nivel, base, etapa, sucesso=False):
    if sucesso:
        titulo = "Sucesso"
        mensagem = f"Extração realizada com sucesso para a base {base} na etapa {etapa}."
    else:
        if nivel == 1:
            titulo = "Alerta Baixo"
        elif nivel == 2:
            titulo = "Alerta Médio"
        elif nivel == 3:
            titulo = "Alerta Alto"
        else:
            titulo = "Alerta Desconhecido"
        
        mensagem = f"Falha no carregamento da base {base} na etapa {etapa};"
    
    data_atual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    notification.notify(
        title=f"ATENÇÃO: {titulo}",
        message=f"{mensagem}\n{data_atual}",
        app_name='API_NoobMaster',
        timeout=10
    )

# URLs das APIs
urls = {
    "Icelandic": "https://restcountries.com/v3.1/lang/Icelandic",
    "Maltese": "https://restcountries.com/v3.1/lang/Maltese",
    "luxemburguês": "https://restcountries.com/v3.1/lang/Luxembourgish"
}

# Extração dos dados
def extrair_dados(urls):
    tabelas = []  # Para armazenar as tabelas extraídas
    for idioma, url in urls.items():
        try:
            response = requests.get(url)
            response.raise_for_status()
            dados = response.json()
            print(f"Extração bem-sucedida da base {idioma}")
            
            # Notificação de sucesso
            alerta(nivel=1, base=idioma, etapa="EXTRACAO", sucesso=True)
            
            # Transformar os dados em um DataFrame
            df = pd.json_normalize(dados)
            tabelas.append(df)
            
        except requests.exceptions.RequestException as e:
            # Caso ocorra algum erro, gerar alerta
            alerta(nivel=2, base=idioma, etapa="EXTRACAO")
            print(f"Erro na extração da base {idioma}: {e}")

    return tabelas

# Tratamento dos dados
def tratar_dados(tabelas):
    tabelas_tratadas = []  # Lista para armazenar as tabelas tratadas
    for idx, tabela in enumerate(tabelas):
        
        # Colunas que EU QUERO!!!
        colunas_interesse = ['name.common', 'capital', 'area', 'population', 'region', 'subregion']
        tabela_tratada = tabela[colunas_interesse].copy()
        tabela_tratada['capital'] = tabela_tratada['capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

        tabela_tratada = tabela_tratada.rename(columns={
            'name.common': 'Nome',
            'capital': 'Capital',
            'area': 'Área',
            'population': 'População',
            'region': 'Região',
            'subregion': 'Sub-Região'
        })
        
        tabelas_tratadas.append(tabela_tratada)
       

    return tabelas_tratadas



# ***Chamando as funções***

In [12]:
tabelas_extraidas = extrair_dados(urls)
tabelas_tratadas = tratar_dados(tabelas_extraidas)

Extração bem-sucedida da base Icelandic
Extração bem-sucedida da base Maltese
Extração bem-sucedida da base luxemburguês


# ***Exibir primeira tabela***

In [99]:
tabela_1 = tabelas_tratadas[0]
print("\nTabela 1:")
print(tabela_1.head())


Tabela 1:
      Nome    Capital      Área  População  Região       Sub-Região
0  Iceland  Reykjavik  103000.0     366425  Europe  Northern Europe


# ***Exibir segunda tabela***

In [100]:
tabela_2 = tabelas_tratadas[1]
print("\nTabela 2:")
print(tabela_2.head())


Tabela 2:
    Nome   Capital   Área  População  Região       Sub-Região
0  Malta  Valletta  316.0     525285  Europe  Southern Europe


# ***Exibir terceira tabela***

In [101]:
tabela_3 = tabelas_tratadas[2]
print("\nTabela 3:")
print(tabela_3.head())


Tabela 3:
         Nome     Capital    Área  População  Região      Sub-Região
0  Luxembourg  Luxembourg  2586.0     632275  Europe  Western Europe
